Created to plot the output of coherence_test.py. May be edited to plot other versions of similar output from other files.

In [ ]:
import datetime
import os
import pickle
import sys

import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# sys.path.append('../')
sys.path.append(os.path.join(os.path.dirname(""), os.pardir, os.pardir))
import coherence_analysis.utils as f

The section of the Brady's hotspring data we used has some missing time strectches. The next section of code shows the lengths of continuous records and the
associated time of recording.

In [ ]:
CONTINUOUS_IND_RANGES = [
    [0, 2314],
    [2314, 2488],
    [2488, 2740],
]  # for 60s averaging window, 13th, 14th, 15th of March 2016
CONTINUOUS_IND_RANGES = [
    [0, 1157],
    [1157, 1244],
    [1244, 1370],
]  # for 120s averaging window, 13th, 14th of March 2016
CONTINUOUS_IND_RANGES = [
    [0, 1157],
    [1157, 1244],
    [1244, 2089],
]  # for 120s averaging window, 13th, 14th, 15th of March 2016
CONTINUOUS_TIME_RANGES = [
    [
        datetime.datetime(2016, 3, 13, 0, 0, 18),
        datetime.datetime(2016, 3, 14, 14, 34, 48),
    ],
    [
        datetime.datetime(2016, 3, 14, 16, 40, 42),
        datetime.datetime(2016, 3, 14, 19, 35, 12),
    ],
    [
        datetime.datetime(2016, 3, 14, 19, 49, 19),
        datetime.datetime(2016, 3, 14, 23, 59, 19),
    ],
]

In [ ]:
file = r"D:\CSM\Mines_Research\large_scale_coherence_results\coherence_test_results_12_13_2025\svd_metadata_160313000018_160315235949.pkl"
with open(file, "rb") as f:
    metadata = pickle.load(f)

metadata.keys()
metadata["sub_window_length"]
metadata["sampling_rate"]
metadata["files"][-1]
# meta data int array
met = np.array([int(a) for a in metadata["files"]])

intervals = met[1:] - met[:-1]
# plt.hist(intervals, bins=100)
np.where(intervals == np.max(intervals[2889:]))
# intervals[2877:2882] # [2879]
# metadata['files'][4632+342:4632+348]  #[4626:4632]
# metadata["files"][4626:4632]
# plt.plot(intervals[4632+345:4632+375])
# plt.yscale("log")
# intervals[4632+345]
# metadata['files'][4977]

In [ ]:
metadata.keys()

In [ ]:
len(metadata["window_end_times"])
metadata["window_start_times"][1365:1372]

In [ ]:
tick_size = 14
label_size = 16
title_size = 18
legend_size = 12
colors = ["#2c7bb6", "#abd9e9", "#ffffbf", "#fdae61", "#d7191c"]
okabe_ito = [
    "#E69F00",  # orange
    "#56B4E9",  # sky blue
    "#009E73",  # bluish green
    "#F0E442",  # yellow
    "#0072B2",  # blue
    "#D55E00",  # vermillion
    "#CC79A7",  # reddish purple
]
sns.set_theme(style="whitegrid", context="paper", font_scale=1.2)
dpi = 600
save_directory = os.path.join(
    os.path.dirname(""), os.pardir, os.pardir, "data"
)

In [ ]:
# metadata['files']
file_dates = [
    mdates.date2num(datetime.datetime.strptime(f, "%y%m%d%H%M%S"))
    for f in metadata["files"]
]
fig, ax = plt.subplots(figsize=(7, 10))
ax.plot(file_dates, "-o")
ax.yaxis_date()
fig.autofmt_xdate()
# ax.xaxis.set_major_formatter(mdates.DateFormatter("%d-%H:%M:%S"))

Get the catalog

In [ ]:
test_mdates = [
    datetime.datetime(2016, 1, 1, 0, 0, 0),
    datetime.datetime(2016, 1, 1, 0, 2, 0),
]
mds = mdates.date2num(test_mdates)
minimum_interval = mds[1] - mds[0]

In [ ]:
dataframe1 = pd.read_excel(
    r"C:\Users\issah\Downloads\BradyHotspring\catalog\Brady_hotspring_catalog.xlsx"
)
dataframe1 = dataframe1[dataframe1["Parent Template"] == 2201050].reset_index(
    drop=True
)
dataframe1 = dataframe1[dataframe1["Day"] < 16].reset_index(drop=True)
# dataframe1 = dataframe1[dataframe1["Visibility on WF*"]==0].reset_index(drop=True)

# convert dataframe times to datetime
dates = [
    datetime.date(
        dataframe1.iloc[i]["Year"],
        dataframe1.iloc[i]["Mon"],
        dataframe1.iloc[i]["Day"],
    )
    for i in range(len(dataframe1))
]
times = [dataframe1.iloc[i]["Time"] for i in range(len(dataframe1))]
datetimes = [
    datetime.datetime.combine(dates[i], times[i])
    for i in range(len(dataframe1))
]
plot_times = mdates.date2num(datetimes)

In [ ]:
time_intervals = plot_times[1:] - plot_times[:-1]

plot_peaks = [dataframe1.iloc[0]["Peak CC"]] + [
    dataframe1.iloc[i + 1]["Peak CC"]
    for i in range(len(plot_times) - 1)
    if time_intervals[i] > minimum_interval
]
visibility = [dataframe1.iloc[0]["Peak CC"]] + [
    dataframe1.iloc[i + 1]["Visibility on CC*"]
    for i in range(len(plot_times) - 1)
    if time_intervals[i] > minimum_interval
]
plot_times = [plot_times[0]] + [
    plot_times[i]
    for i in range(len(plot_times) - 1)
    if time_intervals[i] > minimum_interval
]

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

im = ax.plot(plot_times, plot_peaks, "ro")

ax.set_title("Catalog of Brady Hotspring Earthquakes")
ax.xaxis_date()
fig.autofmt_xdate()
ax.xaxis.set_major_formatter(mdates.DateFormatter("%d-%H:%M:%S"))

In [ ]:
vibe_dataframe = pd.read_csv(
    r"C:\Users\issah\Downloads\BradyHotspring\metadata\Vibroseis_segy_timingLog_PoroTomo_Mar2016_Phases_1234_wUTM_v4.txt"
)


def _get_vibe_datetime(row):
    try:
        date_str = row["SigRec_segy_filename"][-10:-4]
        date_str = date_str[:2] + ":" + date_str[2:4] + ":" + date_str[4:]
        date_str = row[" DAQ_Date"] + " " + date_str
        out = datetime.datetime.strptime(date_str, "%m/%d/%Y %H:%M:%S")
    except ValueError:
        date_str = row[" DAQ_Date"] + " " + row[" DAQ_Time"]
        out = datetime.datetime.strptime(date_str, "%m/%d/%Y %H:%M:%S")
    return out


vibe_datetimes = [
    _get_vibe_datetime(vibe_dataframe.iloc[i]) for i in range(869, 1682)
]
vibe_plottimes = mdates.date2num(vibe_datetimes)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

plot_points = np.ones(len(vibe_plottimes))

im = ax.plot(vibe_plottimes, plot_points, "ro")

ax.set_title("Vibroseis Data Acquisition Times")
ax.xaxis_date()
fig.autofmt_xdate()
ax.xaxis.set_major_formatter(mdates.DateFormatter("%d-%H:%M:%S"))

In [ ]:
file = r"D:\CSM\Mines_Research\large_scale_coherence_results\coherence_test_results_12_13_2025\qr_detection_significance_160313000018_160315235949.pkl"
with open(file, "rb") as f:
    detection_significances_qr = pickle.load(f)
file = r"D:\CSM\Mines_Research\large_scale_coherence_results\coherence_test_results_12_13_2025\svd_detection_significance_160313000018_160315235949.pkl"
with open(file, "rb") as f:
    detection_significances_svd = pickle.load(f)

# plt.imshow(detection_significances.T[1:], aspect="auto")
# plt.colorbar()
# # detection_significances.shape

freqs = np.linspace(
    0,
    int(metadata["sampling_rate"] / 2),
    detection_significances_qr.shape[1],
)
freq_lims = [freqs[1], freqs[-1]]

In [ ]:
i = 0
for r in CONTINUOUS_IND_RANGES:
    time_lims = mdates.date2num(
        [
            metadata["window_start_times"][r[0]],
            metadata["window_end_times"][r[1] - 1],
        ]
    )
    fig, ax = plt.subplots(figsize=(15, 6))
    log_det_sig = np.log(detection_significances_qr[r[0] : r[1], 1:300])
    im = ax.imshow(
        log_det_sig.T,
        extent=[time_lims[0], time_lims[-1], freq_lims[-1], freq_lims[0]],
        aspect="auto",
        vmax=np.percentile(log_det_sig.T, 97),
        vmin=np.percentile(log_det_sig.T, 10),
        cmap="cividis",
    )
    ax.set_title("Log Detection Significance")
    ax.xaxis.set_ticks(np.linspace(time_lims[0], time_lims[-1], 15))
    fig.colorbar(im)
    ax.xaxis_date()
    fig.autofmt_xdate()
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%d-%H:%M:%S"))
    i += 1

In [ ]:
mean_ds = np.min(detection_significances_qr, axis=1)
mean_rm_ds = detection_significances_qr - mean_ds[:, np.newaxis]
# mean_ds = np.median(mean_rm_ds, axis=0)
# mean_rm_ds = mean_rm_ds - mean_ds[np.newaxis]
i = 0
for r in CONTINUOUS_IND_RANGES:
    # time_lims = mdates.date2num(
    #     [CONTINUOUS_TIME_RANGES[i][0], CONTINUOUS_TIME_RANGES[i][1]]
    # )
    time_lims = mdates.date2num(
        [
            metadata["window_start_times"][r[0]],
            metadata["window_start_times"][r[1] - 1],
        ]
    )
    fig, ax = plt.subplots(figsize=(15, 6))
    # log_det_sig = np.log(mean_rm_ds[r[0]:r[1],1:])
    log_det_sig = mean_rm_ds[r[0] : r[1], :300]
    im = ax.imshow(
        log_det_sig.T,
        extent=[time_lims[0], time_lims[-1], freq_lims[-1], freq_lims[0]],
        aspect="auto",
        cmap="cividis",
        vmax=np.percentile(log_det_sig.T, 99),
        vmin=np.percentile(log_det_sig.T, 5),
    )  # , vmax=np.percentile(log_det_sig.T, 97)), vmin=np.percentile(log_det_sig.T, 1), cmap="cividis")
    ax.set_title("Log Detection Significance")
    ax.xaxis.set_ticks(np.linspace(time_lims[0], time_lims[-1], 15))
    fig.colorbar(im)
    ax.xaxis_date()
    fig.autofmt_xdate()
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%d-%H:%M:%S"))
    i += 1

In [ ]:
mean_ds = np.min(detection_significances_svd, axis=1)
mean_rm_ds = detection_significances_svd - mean_ds[:, np.newaxis]
# mean_ds = np.median(mean_rm_ds, axis=0)
# mean_rm_ds = mean_rm_ds - mean_ds[np.newaxis]
i = 0
for r in CONTINUOUS_IND_RANGES:
    # time_lims = mdates.date2num(
    #     [CONTINUOUS_TIME_RANGES[i][0], CONTINUOUS_TIME_RANGES[i][1]]
    # )
    time_lims = mdates.date2num(
        [
            metadata["window_start_times"][r[0]],
            metadata["window_start_times"][r[1] - 1],
        ]
    )
    fig, ax = plt.subplots(figsize=(15, 6))
    # log_det_sig = np.log(mean_rm_ds[r[0]:r[1],1:])
    log_det_sig = mean_rm_ds[r[0] : r[1], :300]
    im = ax.imshow(
        log_det_sig.T,
        extent=[time_lims[0], time_lims[-1], freq_lims[-1], freq_lims[0]],
        aspect="auto",
        cmap="cividis",
        vmax=np.percentile(log_det_sig.T, 99),
        vmin=np.percentile(log_det_sig.T, 5),
    )  # , vmax=np.percentile(log_det_sig.T, 97)), vmin=np.percentile(log_det_sig.T, 1), cmap="cividis")
    ax.set_title("Log Detection Significance")
    ax.xaxis.set_ticks(np.linspace(time_lims[0], time_lims[-1], 15))
    fig.colorbar(im)
    ax.xaxis_date()
    fig.autofmt_xdate()
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%d-%H:%M:%S"))
    i += 1

In [ ]:
event_050_2 = [704, 838, 841, 865, 923, 934, 977, 979]
event_050_1 = [1041, 1047]
event_050_0 = [791, 704]
others = [799, 806, 831]
obscured_by_vibe = [631, 750]

In [ ]:
plt.figure(figsize=(10, 6), dpi=dpi)
event_050_2 = [704, 838, 841, 865, 923, 934, 977, 979]
event_050_2_plot = [631, 791, 838, 979, 1085]
event_050_2_plot = [704, 865, 923, 791]
event_050_2_plot = [790, 791, 865, 978]
not_in_catalog = [705]
# 799, event id: 490
alpha = 0
signal_labels = ["A", "B", "C", "D"]
for i, ev in enumerate(event_050_2_plot):
    # alpha += 0.1
    alpha = 0.8
    plt.plot(
        freqs[:200],
        mean_rm_ds[ev][:200],
        "-o",
        alpha=alpha,
        label=f"Event {signal_labels[i]}",
    )

plt.ylabel("Detection Parameter", fontsize=label_size)
plt.xlabel("Frequency", fontsize=label_size)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
plt.axvline(22, color="k", linestyle="dashed")
plt.axvline(7, color="k", linestyle="dashed")
# plt.ylim([0, 0.9])
plt.legend(fontsize=legend_size)

In [ ]:
det_par_qr = []
det_par_svd = []
signal_label = []
event_labels = ["A", "B", "C", "D"]
for i, ev in enumerate(event_050_2_plot):
    det_par_qr.extend(detection_significances_qr[ev][:200])
    det_par_svd.extend(detection_significances_svd[ev][:200])
    signal_label.extend([f"Data {event_labels[i]}"] * 200)

In [ ]:
det_dict = {
    "Frequency": list(freqs[:200]) * len(event_labels) * 2,
    "Detection Parameters": det_par_qr + det_par_svd,
    "Event Labels": signal_label * 2,
    "Algorithm": ["2"] * len(det_par_qr) + ["1"] * len(det_par_svd),
}
df_det = pd.DataFrame(det_dict)

In [ ]:
plt.figure(dpi=dpi)
g = sns.FacetGrid(
    df_det,
    col="Algorithm",
    col_order=["1", "2"],
    # hue="Method",
    # style="Data_Label",
    height=5.5,
    aspect=1.2,
    # palette="colorblind",
)
g.map_dataframe(
    sns.lineplot,
    x="Frequency",
    y="Detection Parameters",
    hue="Event Labels",
    style="Event Labels",
    # linewidth=2.5,
    markers=True,
    # markersize=6,
    dashes=False,  # cleaner, optional
    err_style="bars",
    # errorbar=("sd", 2.5),
    errorbar=("pi", 95),
    alpha=0.8,
    palette="colorblind",  # okabe_ito,  #"deep" stronger colors than "pastel"
    # legend=False,
    # title=False,
)

# Axis labels
g.set_axis_labels(
    "Frequency",
    "Detection Parameter",
    #   fontsize=label_size
)

# Facet titles
# g.set_titles(size=label_size)

# Tick label sizes
for ax in g.axes.flat:
    ax.tick_params(
        axis="both",
        #    labelsize=tick_size
    )

# Legend fontsize
# g.add_legend(title="Data Label")
g.add_legend(
    fontsize=legend_size,
    frameon=True,
    # bbox_to_anchor=(0.8, 0.7)
)  # , loc="center left")
# plt.setp(g._legend.get_texts(), fontsize=legend_size)
# plt.setp(g._legend.get_title(), fontsize=13)

plt.text(
    0.06,
    0.9,
    "(a)",
    # fontsize=label_size,
    weight="bold",
    transform=plt.gcf().transFigure,
    bbox=dict(facecolor="white", alpha=0.9, boxstyle="round,pad=0.1"),
)
plt.text(
    0.49,
    0.9,
    "(b)",
    # fontsize=label_size,
    weight="bold",
    transform=plt.gcf().transFigure,
    bbox=dict(facecolor="white", alpha=0.9, boxstyle="round,pad=0.1"),
)

In [ ]:
# ratios = [a/b for a, b in zip(det_par_svd, det_par_qr)]
ratios = (
    detection_significances_svd[:][:200] / detection_significances_qr[:][:200]
)
ratios = ratios.flatten()

In [ ]:
plt.figure(figsize=(10, 6), dpi=dpi)
plt.figure(dpi=dpi)
sns.histplot(
    ratios,
    edgecolor="black",
    alpha=0.6,
    linewidth=1.2,
    bins=75,
    stat="density",
)
# plt.axvline(1.8, color="k", linestyle="dashed")
# sns.kdeplot(ratios, linewidth=2)
# plt.xlabel("SVD to QR Detection Parameter Ratio", fontsize=label_size)
plt.xlabel(r"$\frac{\theta}{\hat{\theta}}$", fontsize=label_size * 1.5)
plt.ylabel("Density", fontsize=label_size)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size);

In [ ]:
a = "03/14/16 08:38:18"
datetime.datetime.strptime(a, "%m/%d/%y %H:%M:%S")

In [ ]:
i = 841
print(
    metadata["window_start_times"][i],
    metadata["window_end_times"][i]
    - datetime.timedelta(seconds=120000)
    + datetime.timedelta(seconds=120),
)

In [ ]:
event_050_2 = [838, 841, 865, 923, 934, 977, 979]
event_050_1 = [631, 1041]
event_050_0 = [791, 704]
others = [799, 806, 831]

In [ ]:
mean_ds = np.min(detection_significances_qr, axis=1)
mean_rm_ds = detection_significances_qr - mean_ds[:, np.newaxis]

In [ ]:
plt.figure(figsize=(10, 6), dpi=dpi)
# plt.plot(freqs[:300], mean_rm_ds[979][:300], "--o")
# plt.plot(freqs[:300], mean_rm_ds[1086][:300], "--o")
# plt.plot(freqs[:300], mean_rm_ds[934][:300], "--o")
# plt.plot(freqs[:300], mean_rm_ds[966][:300], "--o")
# plt.plot(freqs[:300], mean_rm_ds[674][:300], "--o")
plt.plot(freqs[:200], mean_rm_ds[790][:200], "--o")
# plt.plot(freqs[5:], mean_rm_ds[2732][5:], "r-o")
# plt.plot(freqs[5:], mean_rm_ds[1330][5:], "k-o")
# plt.plot(freqs[5:], mean_rm_ds[1611][5:], "r-o")
plt.vlines(7.5, 0, 0.1, color="k", linestyle="--")
plt.vlines(17, 0, 0.1, color="k", linestyle="--")

In [ ]:
plt.figure(figsize=(18, 6))
f_ind = 15  # frequency index to plot
mean_ds = np.median(mean_rm_ds, axis=0)
mean_rm_ds = detection_significances_qr - mean_ds[np.newaxis]
plt.plot(mean_rm_ds[:, f_ind], "r-o")

# Define the window size
window_size = 10

# Create a window of ones, normalized to sum to 1
window = np.ones(window_size) / float(window_size)

# Convolve the data with the window
moving_average = np.convolve(mean_rm_ds[:, f_ind], window, "valid")

plt.plot(moving_average, "b-o", alpha=0.5)

In [ ]:
i = 0
threshold = 0.04
last_ind = CONTINUOUS_IND_RANGES[-1][1]
r = [790, 1140]
# average_detection = np.mean(detection_significances_qr[:last_ind, 25:35], axis=1)
average_detection = detection_significances_svd[:last_ind, 17] - np.median(
    detection_significances_svd[:last_ind], axis=1
)
# average_detection = np.mean(
#     detection_significances_qr[:last_ind, 25:45] - np.median(detection_significances_qr[25:45], axis=1)[np.newaxis], axis=1
# )

# average_detection = average_detection - np.median(average_detection)
# average_detection = np.mean(mean_rm_ds[:last_ind,5:15], axis=1)

fig, ax = plt.subplots(figsize=(12, 6))
im = ax.plot(
    metadata["window_start_times"][r[0] : r[1]],
    average_detection[r[0] : r[1]],
    "ko",
    linewidth=2,
    label="detection parameter",
)
plot_peaks = np.array(plot_peaks)
plot_times = np.array(plot_times)
visibility = np.array(visibility)
im1 = ax.plot(
    plot_times[visibility == 1],
    plot_peaks[visibility == 1],
    "ro",
    alpha=0.5,
    label="visibility 1",
)
im1 = ax.plot(
    plot_times[visibility == 2],
    plot_peaks[visibility == 2],
    "bo",
    alpha=0.5,
    label="visibility 2",
)
im1 = ax.plot(
    plot_times[visibility == 0],
    plot_peaks[visibility == 0],
    "yo",
    alpha=0.5,
    label="visibility 0",
)
# im2 = ax.plot(vibe_plottimes, plot_points, "b*", label="vibroseis data acquisition times")
ax.plot(metadata["window_start_times"][695], 0.13, "go")
ax.xaxis_date()
fig.autofmt_xdate()
ax.xaxis.set_major_formatter(mdates.DateFormatter("%d-%H:%M:%S"))
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.ylim(-0.07, 1.05)
plt.xlabel("Time", fontsize=15)
plt.ylabel("Detection parameter", fontsize=15)
plt.legend(fontsize=12)

In [ ]:
file = r"D:\CSM\Mines_Research\large_scale_coherence_results\coherence_test_results_12_13_2025\qr_eig_estimatess_160313000018_160315235949.pkl"
with open(file, "rb") as f:
    eig_estimates_qr = pickle.load(f)
# file = r"D:\CSM\Mines_Research\large_scale_coherence_results\coherence_test_results_12_13_2025\svd_eig_estimatess_160313000018_160315235949.pkl"
# with open(file, "rb") as f:
#     eig_estimates_svd = pickle.load(f)

In [ ]:
eig_estimates_qr = eig_estimates_qr.reshape(
    eig_estimates_qr.shape[0], 2089, 120
)

In [ ]:
event_050_2 = [838, 841, 865, 923, 934, 977, 979]
event_050_1 = []
event_050_0 = [791, 704]
others = [799, 806, 831]
obscured_by_vibe = [631, 750]
plt.figure(figsize=(12, 6), dpi=dpi)
i = 841  # two different events close in time
# i = 1112
i = 978
x_ax = [
    datetime.timedelta(seconds=a) + metadata["window_start_times"][i]
    for a in range(eig_estimates_qr.shape[2])
]
plt.plot(x_ax, eig_estimates_qr[7, i, :] / 500, "r-o")
plt.plot(x_ax, eig_estimates_qr[9, i, :] / 500, "b-o")
# plt.plot(x_ax, eig_estimates_qr[9,i,:] / 500, "y-o")
plt.plot(x_ax, eig_estimates_qr[17, i, :] / 500, "k-o")
plt.vlines(
    datetime.timedelta(seconds=47) + metadata["window_start_times"][i],
    ymin=0,
    ymax=0.2,
    colors="k",
    linestyles="dashed",
)
plt.vlines(
    datetime.timedelta(seconds=52) + metadata["window_start_times"][i],
    ymin=0,
    ymax=0.2,
    colors="k",
    linestyles="dashed",
)
plt.vlines(
    datetime.timedelta(seconds=56) + metadata["window_start_times"][i],
    ymin=0,
    ymax=0.2,
    colors="k",
    linestyles="dashed",
)
plt.vlines(
    datetime.timedelta(seconds=77) + metadata["window_start_times"][i],
    ymin=0,
    ymax=0.2,
    colors="k",
    linestyles="dashed",
)
plt.ylabel("Normalized Eigenvalue", fontsize=label_size)
plt.xlabel("Time", fontsize=label_size)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size);

In [ ]:
eig_estimates_qr.shape

In [ ]:
metadata["window_start_times"][i]

In [ ]:
i = 985
plt.figure(figsize=(12, 6), dpi=dpi)
x_ax = [
    datetime.timedelta(seconds=a) + metadata["window_start_times"][i]
    for a in range(eig_estimates_qr.shape[2])
]
plt.plot(x_ax, eig_estimates_qr[7, i, :] / 500, "-o")
# plt.plot(x_ax, eig_estimates_qr[8,i,:] / 500, "-o")
plt.plot(x_ax, eig_estimates_qr[9, i, :] / 500, "-o")
plt.plot(x_ax, eig_estimates_qr[15, i, :] / 500, "-o")
# plt.ylim(-0.01, 0.2)

In [ ]:
metadata["files"].index("160314083518")

In [ ]:
np.array(range(70))[50 : 65 : int(15 / 5)]

In [ ]:
np.arange(50, 65, 3)

In [ ]:
eig_estimates_qr[0, 1, [1, 2]]

In [ ]:
datetime.timedelta(seconds=47) + metadata["window_start_times"][i]

In [ ]:
mdates.num2date(plot_times[22])

In [ ]:
metadata["window_start_times"][662]

In [ ]:
vibe_datetimes[5:]

In [ ]:
i = 0
threshold = 0.02
last_ind = CONTINUOUS_IND_RANGES[-1][1]
# average_detection = np.mean(detection_significances_qr[:last_ind, 25:35], axis=1)
average_detection = detection_significances_qr[:last_ind, 35] - np.median(
    detection_significances_qr[:last_ind], axis=1
)
average_detection = np.mean(
    detection_significances_qr[:last_ind, 25:45]
    - np.median(detection_significances_qr[25:45], axis=1)[np.newaxis],
    axis=1,
)

average_detection = average_detection - np.median(average_detection)
# average_detection = np.mean(mean_rm_ds[:last_ind,5:15], axis=1)

times = np.empty(average_detection.shape[0], dtype=object)
# np.linspace(CONTINUOUS_TIME_RANGES[0][0], CONTINUOUS_TIME_RANGES[0][-1], CONTINUOUS_IND_RANGES[0][  ]- CONTINUOUS_IND_RANGES[0][0])
for r in CONTINUOUS_IND_RANGES:
    # time_lims = mdates.date2num(
    #     [CONTINUOUS_TIME_RANGES[i][0], CONTINUOUS_TIME_RANGES[i][1]]
    # )
    time_lims = mdates.date2num(
        [
            metadata["window_start_times"][r[0]],
            metadata["window_start_times"][r[1] - 1],
        ]
    )
    times[r[0] : r[1]] = np.linspace(
        time_lims[0], time_lims[-1], min(len(average_detection), r[1]) - r[0]
    )
    i += 1

fig, ax = plt.subplots(figsize=(12, 6))
im = ax.plot(
    times,
    average_detection,
    "ko",
    linewidth=2,
    label="detection parameter",
)
im1 = ax.plot(
    plot_times, plot_peaks, "ro", alpha=0.5, label="microseismic events"
)
im2 = ax.plot(
    vibe_plottimes, plot_points, "b*", label="vibroseis data acquisition times"
)
ax.plot(times[1965], 0.4, "go")
ax.xaxis_date()
fig.autofmt_xdate()
ax.xaxis.set_major_formatter(mdates.DateFormatter("%d-%H:%M:%S"))
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.ylim(-0.07, 1.05)
plt.xlabel("Time", fontsize=15)
plt.ylabel("Detection parameter", fontsize=15)
plt.legend(fontsize=12)

fig, ax = plt.subplots(figsize=(12, 6))
# log_det_sig = np.log(detection_significances[r[0]:r[1],1:])

# plt.plot(np.linspace(time_lims[0], average_detection[r[0]:r[1]], time_lims[-1], r[1]-r[0]), color="black", linewidth=2)
im = ax.plot(
    times[average_detection > threshold],
    average_detection[average_detection > threshold] - threshold,
    "o",
    color="black",
    linewidth=2,
)
im1 = ax.plot(plot_times, plot_peaks, "ro", alpha=0.5)
im2 = ax.plot(vibe_plottimes, plot_points, "b*")
ax.set_title("Detection parameter", fontsize=12)
# ax.xaxis.set_ticks(np.linspace(time_lims[0], time_lims[-1], 15))
# fig.colorbar(im)
ax.xaxis_date()
fig.autofmt_xdate()
ax.xaxis.set_major_formatter(mdates.DateFormatter("%d-%H:%M:%S"))

In [ ]:
i = 0
threshold = 0.06
last_ind = CONTINUOUS_IND_RANGES[-1][1]
# average_detection = np.mean(detection_significances_qr[:last_ind, 25:35], axis=1)
average_detection = detection_significances_qr[:last_ind, 35] - np.median(
    detection_significances_qr[:last_ind], axis=1
)
average_detection = np.mean(
    detection_significances_qr[:last_ind, 40:60], axis=1
) - np.median(detection_significances_qr[:last_ind], axis=1)

average_detection = average_detection - np.median(average_detection)
# average_detection = np.mean(mean_rm_ds[:last_ind,5:15], axis=1)

times = np.empty(average_detection.shape[0], dtype=object)
# np.linspace(CONTINUOUS_TIME_RANGES[0][0], CONTINUOUS_TIME_RANGES[0][-1], CONTINUOUS_IND_RANGES[0][  ]- CONTINUOUS_IND_RANGES[0][0])
for r in CONTINUOUS_IND_RANGES:
    time_lims = mdates.date2num(
        [CONTINUOUS_TIME_RANGES[i][0], CONTINUOUS_TIME_RANGES[i][1]]
    )
    times[r[0] : r[1]] = np.linspace(
        time_lims[0], time_lims[-1], min(len(average_detection), r[1]) - r[0]
    )
    i += 1

fig, ax = plt.subplots(figsize=(12, 6))
im = ax.plot(
    times[1600:],
    average_detection[1600:],
    "ko",
    linewidth=2,
    label="detection parameter",
)
im1 = ax.plot(
    plot_times, plot_peaks, "ro", alpha=0.5, label="microseismic events"
)
# im2 = ax.plot(vibe_plottimes, plot_points, "b*", label="vibroseis data acquisition times")
ax.plot(times[2709], 0.06, "go")
ax.xaxis_date()
fig.autofmt_xdate()
ax.xaxis.set_major_formatter(mdates.DateFormatter("%d-%H:%M:%S"))
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.ylim(-0.07, 1.05)
plt.xlabel("Time", fontsize=15)
plt.ylabel("Detection parameter", fontsize=15)
plt.legend(fontsize=12)

In [ ]:
without_vibe = list(range(120, 886)) + list(
    range(1581, len(detection_significances_qr[:, 35]))
)
# detection_significances_qr[without_vibe, 35]
plot_qr = detection_significances_qr[:, 35]
plot_svd = detection_significances_svd[:, 35]
# plot_qr = detection_significances_qr.flatten()
# plot_svd = detection_significances_svd.flatten()

plt.figure(figsize=(15, 13))
plt.subplot(221)
plt.hist(plot_qr, bins=150, label="qr")
plt.hist(plot_svd, bins=150, alpha=0.5, label="standard")
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend(fontsize=12)
plt.subplot(222)
plt.hist(plot_qr, bins=100, label="qr")
plt.hist(plot_svd, bins=100, alpha=0.5, label="standard")
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend(fontsize=12)
plt.yscale("log")

plot_qr = detection_significances_qr[without_vibe, 35]
plot_svd = detection_significances_svd[without_vibe, 35]
plt.subplot(223)
plt.hist(plot_qr, bins=100, label="qr")
plt.hist(plot_svd, bins=100, alpha=0.5, label="standard")
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend(fontsize=12)
plt.subplot(224)
plt.hist(plot_qr, bins=100, label="qr")
plt.hist(plot_svd, bins=100, alpha=0.5, label="standard")
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend(fontsize=12)
plt.yscale("log")